# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print("OpenAI API KEY Exist")
else:
    print("Not exist")

MODEL = "gpt-4o-mini"
openai=OpenAI()

OpenAI API KEY Exist


In [3]:
system_message = """You are the customer support assistant for TechShop (an e-commerce site). Your name is Abigail.
Your Tasks:
Check order statuses
Initiate returns/exchanges
Provide product information
Track shipments
Answer general customer questions

Behavior Rules:
Always be friendly and helpful.
Address the customer by their name once you know it.
Show empathy when there is an issue: "I understand — I can imagine this is frustrating for you."
Order numbers must be 10 characters (e.g., ORD1234567).
Shipment tracking numbers must be 12 characters (e.g., KRG123456789012).
Returns must be requested within 14 days of delivery.
Customer satisfaction is your highest priority.

Communication Style:
Speak English; be polite but warm.
You may use emojis sparingly (only where appropriate).
Keep answers short and clear.
If needed, explain steps one-by-one.

Important Info:
Working hours: 09:00–22:00 (we are not 24/7).
Contact: destek@techshop.com or +90 850 123 45 67
Free shipping for orders over 500 TL.
Return period: 14 days from delivery.

If a request falls outside your capability, explain politely and offer an alternative.""" 

In [4]:
def chat(message,history):
    messages = [{"role" : "system" ,"content" : system_message}] + history + [{"role":"user","content":message}]
    response = openai.chat.completions.create(model=MODEL,messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat,type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [5]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "check_order",
            "description" : "Fetches order status, shipment info and product details by order number",
            "parameters" : {
                "type" : "object",
                "properties" : {
                    "order_number": {
                        "type" :"string",
                        "description": "10-character order number (e.g., ORD1234567)",
                        "pattern" : "^[A-Z]{3}\\d{7}$",
                        "minLength": 10,
                        "maxLength":10
                    }
                },
                "required" :["order_number"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name" : "initiate_return",
            "description" : "Starts a return process for the specified order",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_number": {
                        "type" : "string",
                        "description" : "10-character order number (e.g., ORD1234567)",
                        "pattern" : "^[A-Z]{3}\\d{7}$",
                        "minLength" :10,
                        "maxLength" :10
                    },
                    "reason": {
                        "type" : "string",
                        "description" : " Reason for return (e.g., 'damaged', 'wrong item', 'not satisfied')"
                        
                },
                "details":{
                    "type":"string",
                    "description" : "Detailed explanation (optional)"
            }
        },
                "required":["order_number","reason"]
            }
        }
    },
    {
        "type":"function",
        "function" : {
            "name" : "track_shipment",
      "description": "Shows shipment status and last known location using tracking number",
      "parameters": {
        "type": "object",
        "properties": {
          "tracking_number": {
            "type": "string",
            "description": "12-character shipment tracking number (e.g., KRG123456789)",
            "pattern": "^[A-Z]{3}\\d{9}$",
            "minLength": 12,
            "maxLength": 12
          }
        },
        "required": ["tracking_number"]
      }
    }
  },
    {
        "type": "function",
        "function": {
            "name" : "product_info",
            "description" : "Retrieces product details(price,stock,specs) by product name or code",
            "paramters": {
                "type" : "object",
                "properties": {
                    "product_query": {
                        "type" : "string",
                        "description": "Product name or code"
                    }
                },
                "required": ["product_query"]
            }
        }
    },
    {"type": "function",
     "function": {
      "name": "get_faq",
      "description": "Fetches FAQ entries from the database by topic",
      "parameters": {
        "type": "object",
        "properties": {
          "topic": {
            "type": "string",
            "description": "FAQ topic",
            "enum": ["shipping", "payment", "returns", "warranty", "membership"]
          }
        },
        "required": ["topic"]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "create_support_ticket",
      "description": "Creates a support ticket for issues that can't be resolved immediately",
      "parameters": {
        "type": "object",
        "properties": {
          "full_name": {
            "type": "string",
            "description": "Customer full name"
          },
          "email": {
            "type": "string",
            "description": "Contact email (e.g., user@example.com)",
            "format": "email"
          },
          "subject": {
            "type": "string",
            "description": "Ticket subject"
          },
          "description": {
            "type": "string",
            "description": "Detailed description"
          }
        },
        "required": ["full_name", "email", "subject", "description"]
      }
    }
  }
]

In [6]:
import random
from datetime import datetime, timedelta

def check_order(order_number):
    """Fetches order information(simulated)."""
    statues = ["Preparing", "Shipped","Out for Delivery", "Delivered"]
    status = random.choice(statues)
    
    products = [
        {"name": "iPhone 15 Pro", "quantity": 1, "price": 45999},
        {"name": "AirPods Pro", "quantity": 1, "price": 8999},
        {"name": "Apple Watch Series 9", "quantity": 1, "price": 15999}
    ]
    
    product = random.choice(products)

    return {
    "success": True,
    "order_number" : order_number,
    "date": (datetime.now() - timedelta(days=random.randint(1,10))).strftime("%Y-%m-%d"),
    "status" : status,
    "products" : [product],
    "total" : product["price"],
    "shipping_provider" : random.choice(["Aras Kargo", "Yurtici Kargo", "Mng Kargo"]),
    # tracking_number format: 3 letters + 9 digits = 12 chars (matches tools pattern)
    "tracking_number": f"KRG{random.randint(100000000, 999999999)}",
    "estimated_delivery": (datetime.now() + timedelta(days=random.randint(1, 3))).strftime("%Y-%m-%d")
    }

In [7]:
def initiate_return(order_number,reason,details=""):
    """Starts a return process (simulated)."""

    return_id = f"RET{random.randint(100000,999999)}"

    return {
    "success" :True,
    "return_id" : return_id,
    "order_number" : order_number,
    "reason": reason,
    "status" : "Return Request Received",
    "message" : "Your return request has been created successfully.The courier will pick up the item within 2 business days.",
    "return_code": return_id,
    "shipping_fee": "Free if issue caused by us",
    "refund_time": "5-7 business days after the item arrives at the warehouse"
    }

In [8]:
def track_shipment(tracking_number):
    """ Track shipment status"""

    locations = [
        "Istanbul - Transfer Center",
        "Ankara - Transfer Hub",
        "Izmir - Branch",
        "Bursa - Distribution Center"
    ]

    now = datetime.now()
    activities = [
        {"timestamp": (now - timedelta(days=2)).strftime("%Y-%m-%d %H:%M"), "status": "Shipped", "location": "Istanbul - Central"}, #"2025-10-05 23:58" tarih boyle olur mesela
        {"timestamp": (now - timedelta(days=1)).strftime("%Y-%m-%d %H:%M"), "status": "At transfer center", "location": random.choice(locations)},
        {"timestamp": now.strftime("%Y-%m-%d %H:%M"), "status": "Out for delivery", "location": random.choice(locations)}
    ]

    return {
        "success" : True,
        "tracking_number" : tracking_number,
        "last_locations" : random.choice(locations),
        "activites" : activites,
        "estimated_delivery" : (now + timedelta(hours=6)).strftime("%Y-%m-%d %H:%M")
    }    

In [11]:
def product_info(product_query):
    """ Retrieves the product details"""

    product_db = {
        "iphone": {
            "name": "iPhone 15 Pro 256GB",
            "price": 45999,
            "stock": "In stock",
            "features": ["A17 Pro chip", "Titanium frame", "USB-C", "48MP camera"],
            "warranty": "2-year Apple warranty",
            "shipping": "Free shipping"
        },
        "airpods": {
            "name": "AirPods Pro (2nd Gen)",
            "price": 8999,
            "stock": "In stock",
            "features": ["Active Noise Cancellation", "Wireless charging", "H2 chip"],
            "warranty": "1-year Apple warranty",
            "shipping": "Free shipping"
        },
        "macbook": {
            "name": "MacBook Air M2",
            "price": 35999,
            "stock": "Limited stock",
            "features": ["M2 chip", "8GB RAM", "256GB SSD", "13.6 inch"],
            "warranty": "2-year warranty",
            "shipping": "Free shipping"
        }
    }


    #Basic search (in real app use better matching)

    for key,product in product_db.items():
        if key in product_query.lower():
            return {"success": True,**product}
            #** operatörü, bir sözlüğün içindeki tüm anahtar-değer çiftlerini alır ve sanki elle yazılmış gibi başka bir sözlüğün içine "dağıtır" veya "açar".
    return {"success": False, "message": "Product not found"}
    

In [14]:
def get_faq(topic):
    """ Returns FAQ info by topic"""
    faq_db = {
        "shipping": {
            "title": "Shipping and Delivery",
            "questions": [
                "Free shipping for orders over 500 TRY",
                "Delivery time is 1-3 business days",
                "You will receive an SMS when your order is shipped"
            ]
        },
        "payment": {
            "title": "Payment Options",
            "questions": [
                "We accept credit and debit cards",
                "Installment plans are available",
                "Cash on delivery is not available"
            ]
        },
        "returns": {
            "title": "Returns and Exchanges",
            "questions": [
                "You have the right to return within 14 days",
                "Item must be unused and in original packaging",
                "Return shipping fee is covered by us if the issue is our fault"
            ]
        },
        "warranty": {
            "title": "Warranty",
            "questions": [
                "All products come with official distributor warranty",
                "Warranty period varies between 1-2 years depending on product",
                "Invoice is required for warranty procedures"
            ]
        },
        "membership": {
            "title": "Account and Membership",
            "questions": [
                "You can shop without registering",
                "Membership makes order tracking easier",
                "Use 'Forgot password' to reset your password"
            ]
        }
    }
    
    return faq_db.get(topic, {"message": "No information found for this topic"})

In [17]:
def create_support_ticket(full_name,email,subject,description):
    """ Creates a support ticket."""
    ticket_id = f"TKT{random.randint(100000,999999)}"

    return {
    "success" : True,
    "ticket_id" : ticket_id,
    "full_name" : full_name,
    "email": email,
    "subject" :subject,
    "status" : "Open",
    "message" : f"Your support ticket #{ticket_id} has been created.",
    "estimated_response": "Within 24 hours"
    }

In [20]:
import json
from typing import Dict, Any, Optional, List
from enum import Enum

# First, let's define our tool types as an Enum for better type safety
class ToolType(Enum):
    CHECK_ORDER = "check_order"
    INITIATE_RETURN = "initiate_return"
    TRACK_SHIPMENT = "track_shipment"
    PRODUCT_INFO = "product_info"
    GET_FAQ = "get_faq"
    CREATE_SUPPORT_TICKET = "create_support_ticket"

# Tool definitions with their required and optional parameters
TOOL_DEFINITIONS = {
    ToolType.CHECK_ORDER: {
        "name": "check_order",
        "description": "Check the status and details of an order",
        "required_params": ["order_number"],
        "optional_params": []
    },
    ToolType.INITIATE_RETURN: {
        "name": "initiate_return",
        "description": "Start a product return process",
        "required_params": ["order_number", "reason"],
        "optional_params": ["details"]
    },
    ToolType.TRACK_SHIPMENT: {
        "name": "track_shipment",
        "description": "Track the current status of a shipment",
        "required_params": ["tracking_number"],
        "optional_params": []
    },
    ToolType.PRODUCT_INFO: {
        "name": "product_info",
        "description": "Get detailed information about a product",
        "required_params": ["product_query"],
        "optional_params": []
    },
    ToolType.GET_FAQ: {
        "name": "get_faq",
        "description": "Retrieve FAQ information by topic",
        "required_params": ["topic"],
        "optional_params": []
    },
    ToolType.CREATE_SUPPORT_TICKET: {
        "name": "create_support_ticket",
        "description": "Create a new customer support ticket",
        "required_params": ["full_name", "email", "subject", "description"],
        "optional_params": []
    }
}

class ToolCallHandler:
    """
    Main handler class for processing tool calls in the e-commerce system.
    This class manages the execution of various customer service functions.
    """
    
    def __init__(self):
        """Initialize the handler with function mappings."""
        # Map tool names to actual functions
        self.tool_functions = {
            ToolType.CHECK_ORDER: check_order,
            ToolType.INITIATE_RETURN: initiate_return,
            ToolType.TRACK_SHIPMENT: track_shipment,
            ToolType.PRODUCT_INFO: product_info,
            ToolType.GET_FAQ: get_faq,
            ToolType.CREATE_SUPPORT_TICKET: create_support_ticket
        }
        
        # Track call history for debugging and analytics
        self.call_history = []
        
    def validate_parameters(self, tool_type: ToolType, parameters: Dict[str, Any]) -> tuple[bool, Optional[str]]:
        """
        Validate that all required parameters are present for a tool call.
        
        Args:
            tool_type: The type of tool being called
            parameters: The parameters provided for the tool
            
        Returns:
            Tuple of (is_valid, error_message)
        """
        tool_def = TOOL_DEFINITIONS[tool_type]
        
        # Check for missing required parameters
        missing_params = []
        for param in tool_def["required_params"]:
            if param not in parameters or parameters[param] is None:
                missing_params.append(param)
        
        if missing_params:
            return False, f"Missing required parameters: {', '.join(missing_params)}"
        
        # Validate parameter types and values
        validation_errors = self._validate_parameter_values(tool_type, parameters)
        if validation_errors:
            return False, validation_errors
            
        return True, None
    
    def _validate_parameter_values(self, tool_type: ToolType, parameters: Dict[str, Any]) -> Optional[str]:
        """
        Validate parameter values based on tool-specific rules.
        
        Args:
            tool_type: The type of tool being called
            parameters: The parameters to validate
            
        Returns:
            Error message if validation fails, None otherwise
        """
        # Tool-specific validation rules
        if tool_type == ToolType.CHECK_ORDER:
            order_num = parameters.get("order_number", "")
            if not isinstance(order_num, (str, int)):
                return "Order number must be a string or integer"
            if str(order_num).strip() == "":
                return "Order number cannot be empty"
                
        elif tool_type == ToolType.TRACK_SHIPMENT:
            tracking_num = parameters.get("tracking_number", "")
            if not isinstance(tracking_num, str):
                return "Tracking number must be a string"
            # Validate tracking number format (3 letters + 9 digits)
            if len(tracking_num) != 12 or not (tracking_num[:3].isalpha() and tracking_num[3:].isdigit()):
                return "Invalid tracking number format. Expected format: 3 letters + 9 digits (e.g., KRG123456789)"
                
        elif tool_type == ToolType.CREATE_SUPPORT_TICKET:
            email = parameters.get("email", "")
            if not isinstance(email, str) or "@" not in email:
                return "Invalid email address format"
            
            # Check email has basic structure
            parts = email.split("@")
            if len(parts) != 2 or "." not in parts[1]:
                return "Invalid email address format"
                
        elif tool_type == ToolType.GET_FAQ:
            topic = parameters.get("topic", "")
            valid_topics = ["shipping", "payment", "returns", "warranty", "membership"]
            if topic not in valid_topics:
                return f"Invalid topic. Valid topics are: {', '.join(valid_topics)}"
                
        elif tool_type == ToolType.INITIATE_RETURN:
            reason = parameters.get("reason", "")
            if not isinstance(reason, str) or len(reason.strip()) < 3:
                return "Return reason must be at least 3 characters long"
                
        return None
    
    def handle_tool_call(self, tool_name: str, parameters: Dict[str, Any]) -> Dict[str, Any]:
        """
        Main method to handle incoming tool calls.
        
        Args:
            tool_name: Name of the tool to call
            parameters: Parameters for the tool
            
        Returns:
            Dictionary containing the result or error information
        """
        # Record the call attempt
        call_record = {
            "timestamp": datetime.now().isoformat(),
            "tool_name": tool_name,
            "parameters": parameters,
            "status": "pending"
        }
        
        try:
            # Step 1: Validate tool name
            try:
                tool_type = ToolType(tool_name)
            except ValueError:
                call_record["status"] = "failed"
                call_record["error"] = f"Unknown tool: {tool_name}"
                self.call_history.append(call_record)
                return {
                    "success": False,
                    "error": f"Unknown tool: {tool_name}",
                    "available_tools": [t.value for t in ToolType]
                }
            
            # Step 2: Validate parameters
            is_valid, error_message = self.validate_parameters(tool_type, parameters)
            if not is_valid:
                call_record["status"] = "failed"
                call_record["error"] = error_message
                self.call_history.append(call_record)
                return {
                    "success": False,
                    "error": error_message,
                    "tool": tool_name,
                    "provided_parameters": list(parameters.keys()),
                    "required_parameters": TOOL_DEFINITIONS[tool_type]["required_params"]
                }
            
            # Step 3: Get the function and prepare parameters
            func = self.tool_functions[tool_type]
            tool_def = TOOL_DEFINITIONS[tool_type]
            
            # Filter parameters to only include valid ones
            filtered_params = {}
            all_params = tool_def["required_params"] + tool_def["optional_params"]
            for param in all_params:
                if param in parameters:
                    filtered_params[param] = parameters[param]
            
            # Step 4: Execute the function
            result = func(**filtered_params)
            
            # Step 5: Process and return the result
            call_record["status"] = "success"
            call_record["result"] = result
            self.call_history.append(call_record)
            
            # Add metadata to successful responses
            if isinstance(result, dict):
                result["_metadata"] = {
                    "tool_used": tool_name,
                    "timestamp": datetime.now().isoformat(),
                    "call_id": len(self.call_history)
                }
            
            return result
            
        except Exception as e:
            # Handle unexpected errors
            call_record["status"] = "error"
            call_record["error"] = str(e)
            self.call_history.append(call_record)
            
            return {
                "success": False,
                "error": f"Tool execution failed: {str(e)}",
                "tool": tool_name,
                "error_type": type(e).__name__
            }
    
    def batch_handle_tool_calls(self, tool_calls: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """
        Handle multiple tool calls in sequence.
        
        Args:
            tool_calls: List of tool call dictionaries, each containing 'tool_name' and 'parameters'
            
        Returns:
            List of results for each tool call
        """
        results = []
        for call in tool_calls:
            if not isinstance(call, dict):
                results.append({
                    "success": False,
                    "error": "Invalid tool call format. Expected dictionary with 'tool_name' and 'parameters'"
                })
                continue
                
            tool_name = call.get("tool_name")
            parameters = call.get("parameters", {})
            
            if not tool_name:
                results.append({
                    "success": False,
                    "error": "Missing 'tool_name' in tool call"
                })
                continue
            
            result = self.handle_tool_call(tool_name, parameters)
            results.append(result)
            
        return results
    
    def get_call_history(self, limit: int = 10) -> List[Dict[str, Any]]:
        """
        Get the history of recent tool calls.
        
        Args:
            limit: Maximum number of recent calls to return
            
        Returns:
            List of recent tool call records
        """
        return self.call_history[-limit:] if self.call_history else []
    
    def clear_history(self):
        """Clear the call history."""
        self.call_history = []
    
    def get_available_tools(self) -> Dict[str, Dict[str, Any]]:
        """
        Get information about all available tools.
        
        Returns:
            Dictionary with tool information
        """
        tools = {}
        for tool_type in ToolType:
            tool_def = TOOL_DEFINITIONS[tool_type]
            tools[tool_type.value] = {
                "description": tool_def["description"],
                "required_parameters": tool_def["required_params"],
                "optional_parameters": tool_def["optional_params"]
            }
        return tools


# Example usage and testing
if __name__ == "__main__":
    # Create handler instance
    handler = ToolCallHandler()
    
    # Example 1: Check an order
    print("Example 1: Checking an order")
    result = handler.handle_tool_call(
        tool_name="check_order",
        parameters={"order_number": "ORD123456"}
    )
    print(json.dumps(result, indent=2))
    print("\n" + "="*50 + "\n")
    
    # Example 2: Track a shipment
    print("Example 2: Tracking a shipment")
    result = handler.handle_tool_call(
        tool_name="track_shipment",
        parameters={"tracking_number": "KRG123456789"}
    )
    print(json.dumps(result, indent=2))
    print("\n" + "="*50 + "\n")
    
    # Example 3: Handle invalid tool call
    print("Example 3: Invalid tool call")
    result = handler.handle_tool_call(
        tool_name="invalid_tool",
        parameters={}
    )
    print(json.dumps(result, indent=2))
    print("\n" + "="*50 + "\n")
    
    # Example 4: Missing parameters
    print("Example 4: Missing required parameters")
    result = handler.handle_tool_call(
        tool_name="create_support_ticket",
        parameters={"email": "customer@example.com"}  # Missing other required params
    )
    print(json.dumps(result, indent=2))
    print("\n" + "="*50 + "\n")
    
    # Example 5: Batch processing
    print("Example 5: Batch tool calls")
    batch_calls = [
        {
            "tool_name": "get_faq",
            "parameters": {"topic": "shipping"}
        },
        {
            "tool_name": "product_info",
            "parameters": {"product_query": "iphone"}
        }
    ]
    results = handler.batch_handle_tool_calls(batch_calls)
    for i, result in enumerate(results, 1):
        print(f"Call {i} result:")
        print(json.dumps(result, indent=2))
        print()
    
    # Show available tools
    print("\n" + "="*50)
    print("Available tools:")
    tools = handler.get_available_tools()
    for tool_name, info in tools.items():
        print(f"\n{tool_name}:")
        print(f"  Description: {info['description']}")
        print(f"  Required: {info['required_parameters']}")
        print(f"  Optional: {info['optional_parameters']}")

Example 1: Checking an order
{
  "success": true,
  "order_number": "ORD123456",
  "date": "2025-10-03",
  "status": "Preparing",
  "products": [
    {
      "name": "AirPods Pro",
      "quantity": 1,
      "price": 8999
    }
  ],
  "total": 8999,
  "shipping_provider": "Aras Kargo",
  "tracking_number": "KRG981131082",
  "estimated_delivery": "2025-10-08",
  "_metadata": {
    "tool_used": "check_order",
    "timestamp": "2025-10-06T10:26:18.814654",
    "call_id": 1
  }
}


Example 2: Tracking a shipment
{
  "success": false,
  "error": "Tool execution failed: name 'activites' is not defined",
  "tool": "track_shipment",
  "error_type": "NameError"
}


Example 3: Invalid tool call
{
  "success": false,
  "error": "Unknown tool: invalid_tool",
  "available_tools": [
    "check_order",
    "initiate_return",
    "track_shipment",
    "product_info",
    "get_faq",
    "create_support_ticket"
  ]
}


Example 4: Missing required parameters
{
  "success": false,
  "error": "Missing requ

In [21]:
import json
import logging
from datetime import datetime

# varsayılan: TOOL_DEFINITIONS ve ToolCallHandler sınıfın yukarıda tanımlı ve import edilmiş olmalı
# ve ayrıca check_order, track_shipment ... gibi fonksiyonlar global scope'ta mevcut olmalı.

# handler'ı tek bir örnek olarak oluştur
handler = ToolCallHandler()

def _build_functions_for_model():
    """
    TOOL_DEFINITIONS bazlı basit "functions" listesi üretir.
    Model'e gönderilecek JSON Schema minimal tiplerle (string) oluşturulur.
    """
    functions = []
    for tool_type, defn in TOOL_DEFINITIONS.items():
        # defn: {"name":..., "description":..., "required_params": [...], "optional_params":[...]}
        properties = {}
        for p in defn["required_params"] + defn["optional_params"]:
            # basit: tüm params string kabul ediyoruz; istersen tipleri TOOL_DEFINITIONS'ta genişlet
            properties[p] = {"type": "string", "description": f"{p}"}
        function_schema = {
            "name": defn["name"],
            "description": defn["description"],
            "parameters": {
                "type": "object",
                "properties": properties,
                "required": defn["required_params"],
                "additionalProperties": False
            }
        }
        functions.append(function_schema)
    return functions

def _parse_function_call(choice_message):
    """
    Model yanıtındaki function_call bilgisini çıkarır.
    choice_message : response.choices[0].message (SDK'na göre uyumlu olmalı)
    Dönüş: (function_name, arguments_dict) veya (None, None)
    """
    fc = None
    # bazı SDK'lar choice_message.function_call/choice_message.get("function_call") gibi farklılık gösterir
    try:
        fc = getattr(choice_message, "function_call", None) or choice_message.get("function_call")
    except Exception:
        fc = None

    if not fc:
        return None, None

    # fc yapısı: {"name": "...", "arguments": "json-string"}
    func_name = fc.get("name") if isinstance(fc, dict) else getattr(fc, "name", None)
    raw_args = fc.get("arguments") if isinstance(fc, dict) else getattr(fc, "arguments", None)

    # güvenli parse
    try:
        args = json.loads(raw_args) if raw_args else {}
    except Exception:
        # fallback: ast.literal_eval yerine burada basit: empty dict + log
        logging.exception("Failed to parse function_call arguments JSON")
        args = {}

    return func_name, args

def chat(message: str, history: list):
    """
    Basit chat fonksiyonu — model ile function-calling döngüsünü yönetir.
    - message: kullanıcı mesajı (string)
    - history: mesaj geçmişi; model-ready formatta: list of {"role":..., "content":...}
    Döner: assistant'ın metin cevabı (string)
    """
    # 1) mesaj listesi oluştur
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    # 2) functions listesi hazırla
    functions = _build_functions_for_model()

    # 3) ilk model çağrısı (function calling desteğiyle)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        functions=functions  # model'e hangi fonksiyonların kullanılabilir olduğunu söyle
    )

    choice_message = response.choices[0].message  # SDK'na göre uyumlu olmalı
    # direkt assistant cevabı varsa (fonksiyon çağrısı yok)
    assistant_text = None
    try:
        # bazı SDK'larda .content, bazılarında .get("content")
        assistant_text = getattr(choice_message, "content", None) or choice_message.get("content")
    except Exception:
        assistant_text = None

    # 4) Model function_call döndürdüyse -> handler ile çalıştır, sonra follow-up
    func_name, func_args = _parse_function_call(choice_message)
    if func_name:
        # handler'ı çağır
        try:
            tool_result = handler.handle_tool_call(func_name, func_args)
        except Exception as e:
            logging.exception("Tool handler failed")
            # model'e hata bilgisini function rolü olarak verip final cevap isteyebiliriz
            tool_result = {"success": False, "error": "handler_exception", "details": str(e)}

        # 5) tool_result'u model'e role="function" ile geri gönderip final cevabı al
        # note: bazı platformlar için 'role' burada "tool" olabilir; OpenAI-style için "function"
        messages.append({"role": "assistant", "content": None, "function_call": {"name": func_name, "arguments": json.dumps(func_args)}})
        messages.append({"role": "function", "name": func_name, "content": json.dumps(tool_result, ensure_ascii=False)})

        followup = openai.chat.completions.create(model=MODEL, messages=messages)
        follow_choice = followup.choices[0].message
        final_text = getattr(follow_choice, "content", None) or follow_choice.get("content")
        return final_text

    # 6) fonksiyon çağrısı yoksa direkt assistant_text'i döndür
    return assistant_text


In [22]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.
